Imports, loading DFs

In [65]:
import pandas as pd
import re
import os

ai_df = pd.read_csv('Data/remoteVisibAI.csv')
ai_df[['Date','Time']] = ai_df['Time [UTC]'].str.split(' ', expand=True)
ai_df['Date'] = pd.to_datetime(ai_df['Date'])
ai_df = ai_df.drop(['min visibility unit', 'visibility unit', 'Time [UTC]'], axis=1)
ai_df = ai_df[['Date','Time', 'visibility','min visibility', 'direction visibility']]
ai_df = ai_df.rename(columns={"visibility": "VisObs", "min visibility": "MinVisObs", "direction visibility": "MinVisDirOs"})
c1 = (ai_df['Date'] >= '2019-11-01') & (ai_df['Date'] <= '2020-02-29')
c2 = (ai_df['Time'] >= '05:20:00') & (ai_df['Time'] <= '17:00:00')
ai_df['VisObs'] = ai_df['VisObs'].astype(int)
ai_df = ai_df[c1 & c2]

obs_df = pd.read_csv('Data/remoteVisibObs.csv',encoding='latin',sep=';')
obs_df = obs_df.dropna(subset=['VisObs'])
obs_df[['Date','Time']] = obs_df['Timestamp'].str.split('_', expand=True)
obs_df = obs_df.drop(['Timestamp', 'Note', 'DONE.Who'], axis=1)
obs_df = obs_df[['Date','Time', 'VisObs', 'MinVisObs', 'MinVisDirObs']]
def rearrange_date(date_str,dayFirst):
    match = re.match(r'(\d{1,2})[./](\d{1,2})[./](\d{4})', date_str)
    if match and dayFirst:
        return f"{match.group(3)}-{match.group(2).zfill(2)}-{match.group(1).zfill(2)}"
    elif match and not dayFirst:
        return f"{match.group(3)}-{match.group(1).zfill(2)}-{match.group(2).zfill(2)}"
    else:
        return date_str
obs_df['Date'] = obs_df['Date'].apply(lambda x: rearrange_date(x,dayFirst=False)) 
c3 = (obs_df['Date'] >= '2019-11-01') & (obs_df['Date'] <= '2020-02-29')
c4 = (obs_df['Time'] >= '05:20:00') & (obs_df['Time'] <= '17:00:00')
obs_df = obs_df[c3 & c4]

sen_df = pd.read_csv("Data/nov_feb_2019_SENZOR.csv")
sen_df[['Date','Time']] = sen_df['Time [UTC]'].str.split(' ', expand=True)
sen_df['Date'] = sen_df['Date'].apply(lambda x: rearrange_date(x,dayFirst=True)) 
sen_df = sen_df.drop(['Time [UTC]', 'VIS 2Min Avg [m]'], axis=1)
sen_df = sen_df.rename(columns={"VIS 1Min Avg [m]": "VisObs"})
sen_df = sen_df[['Date','Time', 'VisObs']]
c5 = (sen_df['Time'] >= '05:20:00') & (sen_df['Time'] <= '17:00:00')
sen_df['Time'] = sen_df['Time'].str.slice(stop=-3)
sen_df = sen_df[c5].groupby(['Date','Time'])['VisObs'].mean().reset_index()
sen_df['VisObs'] = sen_df['VisObs'].astype(int)

/tmp/ipykernel_3669/4046556285.py:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ai_df['Date'] = pd.to_datetime(ai_df['Date'])


In [72]:
ai_df.head()

,Date,Time,VisObs,MinVisObs,MinVisDirOs
286,2019-11-01,05:20:00,10001,15.0,E
287,2019-11-01,05:50:00,10001,15.0,E
288,2019-11-01,06:20:00,10001,15.0,E
289,2019-11-01,06:50:00,10001,15.0,E
290,2019-11-01,07:20:00,10001,15.0,E
